In [ ]:
## Set notebook to auto reload updated modules
%load_ext autoreload
%autoreload 2

In [ ]:
from shared.config import SHARED_SETTINGS
from shared.db import get_db_uri, get_engine, get_session_pool

In [ ]:
from shared.domain.weatherapi.weather.current import (CurrentWeatherIn, CurrentWeatherOut, CurrentWeatherRepository, CurrentWeatherModel, CurrentWeatherJSONIn, CurrentWeatherJSONOut, CurrentWeatherJSONModel, CurrentWeatherJSONRepository)

from shared.domain.weatherapi.location import (LocationIn, LocationJSONModel, LocationModel, LocationOut, LocationRepository)

from shared.domain.weatherapi.weather.forecast import ForecastJSONIn, ForecastJSONModel, ForecastJSONOut, ForecastJSONRepository

In [ ]:
from config import SETTINGS, DB_SETTINGS

In [ ]:
import ibis
from ibis import sqlite

In [ ]:
display(DB_SETTINGS)

In [ ]:
db_url = get_db_uri(
    drivername=DB_SETTINGS.get("DB_DRIVERNAME"),
    username=DB_SETTINGS.get("DB_USERNAME"),
    password=DB_SETTINGS.get("DB_PASSWORD"),
    host=DB_SETTINGS.get("DB_HOST"),
    port=DB_SETTINGS.get("DB_PORT"),
    database=DB_SETTINGS.get("DB_DATABASE")
)

In [ ]:
db_engine = get_engine(url=db_url, echo=DB_SETTINGS.get("db_echo", False))

In [ ]:
SessionLocal = get_session_pool(engine=db_engine)

---

In [ ]:
## Get ibis connection
con = ibis.sqlite.connect(DB_SETTINGS.get("DB_DATABASE"))

In [ ]:
## List tables
tables = con.list_tables()
display(tables)

In [ ]:
## Iterate over table names and print schemas
for table_name in tables:
    _schema = con.table(table_name)
    display(_schema)

In [ ]:
date_columns = [
    {
        "table": "weatherapi_current_json",
        "date_col": "created_at"
    },
    {
        "table": "weatherapi_current_weather",
        "date_col": "last_updated"
    },
    {
        "table": "weatherapi_forecast_json",
        "date_col": "created_at"
    },
    {
        "table": "weatherapi_location_json",
        "date_col": "created_at"
    }
]

In [ ]:
sorted_tables = []

In [ ]:
for _dict in date_columns:
    _table = _dict["table"]
    _col = _dict["date_col"]
    
    _sorted = con.table(_table).order_by([ibis.desc(_col)])
    sorted_tables.append({"table_name": _table, "data": _sorted})

In [ ]:
for sorted_t in sorted_tables:
    t_name = sorted_t['table_name']
    t_data = sorted_t['data']
    
    t = con.table(t_name)
    res_df = t.head(10).execute()
    
    display(t_name)
    display(res_df)

---

In [ ]:
filtered_tables = []

In [ ]:
for t_dict in date_columns:
    t_name = t_dict["table"]
    date_col = t_dict["date_col"]
    
    _table = con.table(t_name)
    
    # Cast date column to timestamp type (if needed)
    _table = _table.mutate(
        **{date_col: _table[date_col].cast("timestamp")}
    )
    
    # Truncate datetime to 1 minute precision
    minute_col = _table[date_col].truncate("m")

    # Group by truncated time to aggregate min datetime per group
    grouped = _table.group_by(minute_col).aggregate(oldest=_table[date_col].min())

    # Join to get full rows corresponding to oldest rows per minute
    filtered_table = _table.inner_join(grouped, _table[date_col] == grouped["oldest"]).select(_table.columns)

    filtered_tables.append({"table_name": t_name, "data": filtered_table})


In [ ]:
for _t in filtered_tables:
    t_name = _t["table_name"]
    t_data = _t["data"]
    
    t = con.table(t_name)
    res_df = t.head(10).execute()
    
    display(t_name)
    display(res_df)